# Evaluation Metrics 

According to [Bailando](https://github.com/lisiyao21/Bailando/). 

- Calculate the FID score for the kinetic and geometric features, indicating the quality of the movements.
- Calculate the Div score for the kinetic and geometric features, indicating the difference in distribution between the ground truth and the generated movements. 

## 1. Extract the AIST++ features

Generate these for each set of motions you'd like to evaluate (e.g. the validation set of the motions generated from the evaluation set). GT = ground truth, being the dance motions extracted from the videos. 

In [1]:
import os
import numpy as np
import argparse

from utils.features.kinetic import extract_kinetic_features
from utils.features.manual_new import extract_manual_features

import multiprocessing
import functools

In [2]:
def calc_metrics(file_name, motion_dir, save_dir):

    seq_name = file_name[:-4]

    man_file = os.path.join(save_dir, 'manual_features_new', seq_name+"_manual.npy")
    kin_file = os.path.join(save_dir, 'kinetic_features', seq_name+"_kinetic.npy")
    
    keypoints3d = np.load(os.path.join(motion_dir, file_name)) 

    if not os.path.exists(man_file):
        features = extract_manual_features(keypoints3d)
        np.save(man_file, features)
    else:
        print(f'man file of {seq_name} exists')

    if not os.path.exists(kin_file):
        features = extract_kinetic_features(keypoints3d)
        np.save(kin_file, features)
    else:
        print(f'kin file of {seq_name} exists')

    print (seq_name, "is done")

In [ ]:
save_dir = 'eval-data/gt-features'
motion_dir = '../dataset/SWDance/motions' # NOTE: you can take only the validation ones

os.makedirs(save_dir, exist_ok=True)
os.makedirs(os.path.join(save_dir, 'kinetic_features'), exist_ok=True)
os.makedirs(os.path.join(save_dir, 'manual_features_new'), exist_ok=True)

file_names = [f for f in os.listdir(motion_dir) if not f.startswith("M")]
print(f'motion dir: {motion_dir}, files: {file_names}')

# processing
process = functools.partial(calc_metrics, motion_dir=motion_dir, save_dir=save_dir)
pool = multiprocessing.Pool(12)
pool.map(process, file_names)

--> NOTE: run this for all sets of motions you'd like to evaluate. 

## 2. Calculate the metrics.

In [ ]:
from utils.metrics_new import calc_and_save_feats, quantized_metrics

gt_root = 'eval-data/gt-features'
pred_root = 'eval-data/val-features'
    
print('Calculating and saving features')
calc_and_save_feats(gt_root)
calc_and_save_feats(pred_root)


print('Calculating metrics')
print(gt_root)
print(pred_root)
print(quantized_metrics(pred_root, gt_root))